# 匯入資料到資料庫中

這裡使用的SQL為MariaDB 10，python版本為3.5

MariaDB為MySQL的分支，所以套件使用MySQL的即可  
但官方的套件支援度沒這麼新(2016/09/26時)
* MySQL-python只支援的2.X
* Connector/Python只支援到3.4

故採用第三方套件[PyMySQL](https://github.com/PyMySQL/PyMySQL "PyMySQL")，就可以連接MariaDB 10和python 3.5

In [ ]:
import pymysql
from os import listdir


## 匯入雨量站
key(site+yyyymmddHHMMSS), site, datetime, value

In [ ]:
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='husky', db='mysql',charset='UTF8')


for site in listdir('Pingtung/Precipitation'):
    for file in listdir('Pingtung/Precipitation/' + site):
        
        fin = open('Pingtung/Precipitation/{}/{}'.format(site, file), 'rt' )
        lines = fin.readlines()
        fin.close()
        
        mydata = [{'key' : ''}]
        
        for line in lines:
            data = line.split()
            if len(data) < 8:
                continue
            siteName = data[0]
            date = '{}-{}-{} {}:00:00'.format(data[1][0:4], data[1][4:6], data[1][6:8],'0' + str(int(data[1][8:10]) - 1) if int(data[1][8:10]) - 1 < 10 else str(int(data[1][8:10]) - 1))
            value = None if float(data[7]) < 0 else data[7]
            
            if siteName + data[1] != mydata[-1:][0]['key']:
                mydata.append({'key': siteName + data[1], 'site': siteName, 'date': date, 'value': value })
        
        mydata.pop(0)
        if len(mydata) == 0:
            continue
        try:
            with conn.cursor() as cursor:
                sql = "INSERT INTO `ncu`.`precipitation` (`key`, `site`, `date`, `value`) VALUES (%(key)s, %(site)s, %(date)s, %(value)s);"
                cursor.executemany(sql, mydata)
            conn.commit()
        except:
            print(site, file)
            conn.close()
        
    print(site)
    
conn.close()

## 匯入水位站
key(site+yyyymmddHHMMSS), site, datetime, value

In [ ]:
v1Site = ['12010111','12020111','12020211','12030111','12030211','12030311','12040111','12040211','12040311','12190211','12200211','122013M2','12210111','13010111','13010211','130111M2','13030211','13050111','13050211','130511M2','13100111','13110111','13110411','131111M1','13120211','131211M2','13130311','131312M2','13140111','13140211','131411M3','13150111','13150211','13150311','13150411','131512M2','13160111','13160211','13160212','13160311','13160411','13170111','13180111','13180211','13190111','13190112','13210111','13210112','13280111']

conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='husky', db='mysql', charset='UTF8')


for file in listdir('Pingtung/Well'):
    site = file.split('_')[0]
    yyyy = file.split('_')[1][:4]
    if site in v1Site:
        
        fin = open('Pingtung/Well/{}'.format(file), 'rt' )
        lines = fin.readlines()
        fin.close()
            
        mydata = [{'key' : ''}]
            
        for line in lines:
            data = line.split()
            if len(data) < 3 or len(data[1]) < 8 or len(data[0]) < 10 or yyyy != data[0][:4]:
                continue
            
            ymd = data[0].split('-')
            mhs = data[1].split(':')
            
            key = '{}{}{}{}{}'.format(site, ymd[0], ymd[1], ymd[2], mhs[0])
            date = '{} {}'.format(data[0], data[1])
            value = None if float(data[2]) < 0 else data[2]
            
            if key != mydata[-1:][0]['key']:
                mydata.append({'key': key, 'site': site, 'date': date, 'value': value })
                
        mydata.pop(0)
        if len(mydata) == 0:
            continue
        try:
            with conn.cursor() as cursor:
                sql = "INSERT INTO `ncu`.`well` (`key`, `site`, `date`, `value`) VALUES (%(key)s, %(site)s, %(date)s, %(value)s);"
                cursor.executemany(sql, mydata)
            conn.commit()
        except:
            print(site, file)
            conn.close()
                
        print(file)
conn.close()